In [1]:
import random

from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
pass



Did not use initialization regex that was passed: _context_layer._module.weight_ih.*
Did not use initialization regex that was passed: _context_layer._module.weight_hh.*


In [2]:
occupations_1 = ['supervisor',
'janitor',
'cook',
'mover',
'laborer',
'construction worker',
'chief',
'developer',
'carpenter',
'manager',
'lawyer',
'farmer',
'driver',
'salesperson',
'physician',
'guard',
'analyst',
'mechanic',
'sheriff',
'CEO']

In [17]:
occupations_2 = ['cashier',
'teacher',
'nurse',
'assistant',
'secretary',
'auditor',
'cleaner',
'receptionist',
'clerk',
'counselor',
'designer',
'hairdresser',
'attendant',
'writer',
'housekeeper',
'baker',
'accountant',
'editor',
'librarian',
'tailor']

In [18]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']

In [19]:
#WHY this mapping? subject/object/posssesive use <-> verb? up and down cells

In [20]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [21]:
occupations_1.extend(["technician", "accountant", "engineer"])
occupations_2.extend(["teacher", "librarian", "nurse", "paralegal"])

filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [22]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [23]:
verb_action[verb_list_p1[1]] = [action1, action2]
verb_action[verb_list_p1[2]] = [action1, action2]

In [24]:
action3 = ['work was unsatisfactory', 'performance was not up to the mark',
           'application was approved', 'application was not approved',
           'application was under review', 'work was appreciated', 
           'behaviour was unacceptable', 'project scope had been finalised']

verb_action[verb_list_p2[0]] = [action3]

In [25]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']
verb_action[verb_list_p2[1]] = [action4]

In [26]:
def predict_clusters(sentence):
    output = predictor.predict(document = sentence)
    return output['clusters'], output

In [27]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [28]:
unique_input1_set = set()
unique_input1_error_set = set()

occupation_pair_error = {}

occupation1_error = {}

occupation2_error = {}

verb_error = {}

action_error = {}

occupation_pair_count = {}

occupation1_count = {}

occupation2_count = {}

verb_count = {}

action_count = {}

#add conjuction_count and erorr
conjuction_error = {}
conjuction_count = {}

#add pronoun_type, or pronoun count and errors
# pronoun_type_error = {}
# pronoun_type_count = {}

# pronoun_error = {}
# pronoun_count = {}


In [29]:
#TODO: implement Particle swarm optimization; or an evolutionary feedback driven approach below

In [40]:
err_count = 0
ITERS = 300 #30000


for i in range(ITERS):
    oc1 = random.choice(occupations_1)
    oc2 = random.choice(occupations_2)
    verb = random.choice(list(verb_action.keys()))
    action = random.choice(random.choice(verb_action[verb]))
    pronoun = choose_pronoun_type(verb)
    input1 = ("The " + oc1 + " " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
    input2 = ("The " + oc1 + " " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[1] + " " + action) 
    
    input3 = ("The " + oc1 + " " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
    pred1, _ = predict_clusters(input1)
    pred2, _ = predict_clusters(input2)
    #pred3, _ = predict_clusters(input2)
    pred3, _ = predict_clusters(input3)
    
    
#     if(i > 0) and 
    if (i % 30 == 0):
        print("Unique errors: " + str(len(unique_input1_error_set)))
        print("Unique inputs: " + str(len(unique_input1_set)))
        print("------------------------------")
        
        
    unique_input1_set.add(input1)
    
    update_dict(occupation_pair_count, (oc1, oc2))
    update_dict(occupation1_count, oc1)
    update_dict(occupation2_count, oc2)
    update_dict(verb_count, verb)
    update_dict(action_count, action)
    update_dict(conjuction_count, filler_conjunction[verb])

    

    if not (pred1 == pred2 and pred2 == pred3):
        if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
            if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0]) ):
#         if(True):
                err_count += 1
                
                unique_input1_error_set.add(input1)
                
#                 print(pred1, pred2, pred3)
#                 print(input1)
#                 print(input2)
#                 print(input3)
                
                update_dict(occupation_pair_error, (oc1, oc2))
                update_dict(occupation1_error, oc1)
                update_dict(occupation2_error, oc2)
                update_dict(verb_error, verb)
                update_dict(action_error, action)
                update_dict(conjuction_error, filler_conjunction[verb])



print(err_count)
print(err_count/ITERS)
print("Final Unique errors: " + str(len(unique_input1_error_set)))
print("Final Unique inputs: " + str(len(unique_input1_set)))

Unique errors: 9
Unique inputs: 569
------------------------------
Unique errors: 9
Unique inputs: 599
------------------------------
Unique errors: 10
Unique inputs: 629
------------------------------
Unique errors: 11
Unique inputs: 657
------------------------------
Unique errors: 11
Unique inputs: 686
------------------------------
Unique errors: 12
Unique inputs: 714
------------------------------
Unique errors: 13
Unique inputs: 743
------------------------------
Unique errors: 13
Unique inputs: 771
------------------------------
Unique errors: 14
Unique inputs: 797
------------------------------
Unique errors: 14
Unique inputs: 826
------------------------------
5
0.016666666666666666
Final Unique errors: 14
Final Unique inputs: 855


In [37]:
# print(occupation_pair_count)
# print(occupation1_count)
# print(occupation2_count)
# print(verb_count)
# print(action_count)
# print(conjuction_count)

In [38]:
print(occupation_pair_error)
print(occupation1_error)
print(occupation2_error)
print(verb_error)
print(action_error)
print(conjuction_error)

{('developer', 'baker'): 1, ('construction worker', 'auditor'): 1, ('supervisor', 'auditor'): 1, ('CEO', 'nurse'): 1, ('analyst', 'hairdresser'): 1}
{'developer': 1, 'construction worker': 1, 'supervisor': 1, 'CEO': 1, 'analyst': 1}
{'baker': 1, 'auditor': 2, 'nurse': 1, 'hairdresser': 1}
{'asked the': 2, 'told the': 2, 'disclosed to the': 1}
{'can bake bread': 1, 'can generate the report': 4}
{'if ': 2, 'that ': 3}


In [35]:
import pickle

with open('saved_pickles/Exploration/unique_input1_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_set, handle)
    
with open('saved_pickles/Exploration/unique_input1_error_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_error_set, handle)

FileNotFoundError: [Errno 2] No such file or directory: 'saved_pickles/Exploration/unique_input1_set.pickle'

In [ ]:
with open('saved_pickles/Exploration/occupation_pair_count.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_count, handle)
    
with open('saved_pickles/Exploration/occupation1_count.pickle', 'wb') as handle:
    pickle.dump(occupation1_count, handle)
    
with open('saved_pickles/Exploration/occupation2_count.pickle', 'wb') as handle:
    pickle.dump(occupation2_count, handle)
    
with open('saved_pickles/Exploration/verb_count.pickle', 'wb') as handle:
    pickle.dump(verb_count, handle)

with open('saved_pickles/Exploration/action_count.pickle', 'wb') as handle:
    pickle.dump(action_count, handle)

In [ ]:
with open('saved_pickles/Exploration/occupation_pair_error.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_error, handle)
    
with open('saved_pickles/Exploration/occupation1_error.pickle', 'wb') as handle:
    pickle.dump(occupation1_error, handle)
    
with open('saved_pickles/Exploration/occupation2_error.pickle', 'wb') as handle:
    pickle.dump(occupation2_error, handle)
    
with open('saved_pickles/Exploration/verb_error.pickle', 'wb') as handle:
    pickle.dump(verb_error, handle)

with open('saved_pickles/Exploration/action_error.pickle', 'wb') as handle:
    pickle.dump(action_error, handle)

In [ ]:
#                 if((oc1, oc2) in occupation_pair_error.keys()):
#                     occupation_pair_error[(oc1, oc2)] += 1
#                 else:
#                     occupation_pair_error[(oc1, oc2)] = 1
                                          
#                 if(oc1 in occupation1_error.keys()):
#                     occupation1_error[oc1] += 1
#                 else:
#                     occupation1_error[oc1] = 1
                
#                 if(oc2 in occupation2_error.keys()):
#                     occupation2_error[oc1] += 1
#                 else:
#                     occupation2_error[oc1] = 1
                                          
#                 if(verb in verb_error.keys()):
#                     verb_error[verb] += 1
#                 else:
#                     verb_error[verb] = 1

In [13]:
# def predict_clusters(sentence):
#     output = predictor.predict(document = sentence)
#     return output['clusters'], output

In [51]:
predict_clusters("The guard spoke with the librarian about his struggles with addiction")

([[[0, 1], [7, 7]]],
 {'top_spans': [[0, 1], [4, 5], [7, 7], [7, 10]],
  'antecedent_indices': [[0, 1, 2, 3],
   [0, 1, 2, 3],
   [0, 1, 2, 3],
   [0, 1, 2, 3]],
  'predicted_antecedents': [-1, -1, 0, -1],
  'document': ['The',
   'guard',
   'spoke',
   'with',
   'the',
   'librarian',
   'about',
   'his',
   'struggles',
   'with',
   'addiction'],
  'clusters': [[[0, 1], [7, 7]]]})

In [35]:
# err_count = 0
# ITERS = 20



# for i in range(ITERS):
#     oc1 = random.choice(occupations_1)
#     oc2 = random.choice(occupations_2)
#     verb = random.choice(list(verb_action.keys()))
#     action = random.choice(random.choice(verb_action[verb]))
#     in1 = ("The " + oc1 + " " + verb + " "
#            + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
#     in2 = ("The " + oc1 + " " + verb + " "
#            + oc2 + " " + filler_conjunction[verb] +  pronoun[1] + " " + action) 
    
#     in3 = ("The " + oc1 + " " + verb + " "
#            + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
#     pred1, _ = predict_clusters(in1)
#     pred2, _ = predict_clusters(in2)
#     pred3, _ = predict_clusters(in2)
    
#     if not (pred1 == pred2 and pred2 == pred3):
#         if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
#             err_count += 1

#             print(pred1, pred2, pred3)
#             print(in1)
#             print(in2)
#             print(in3)
#             print()
    

# print(err_count)
# print(err_count/ITERS)